# Basic analysis of secondary structure results

This notebook loads the secondary structure results in `stucture_results_28C.pickle` calculated by `calculate_secondary_structure.ipynb` and runs some basic correlation analysis with RRS and estimated abundances.

In [1]:
import pickle
import sys

import numpy
import matplotlib
from matplotlib import pyplot
import pandas

utils_dir = '../utils'
sys.path.append(utils_dir)
import custom_plots

In [2]:
%matplotlib inline
matplotlib.rcParams['figure.dpi'] = 120
matplotlib.rcParams['font.sans-serif'] = 'Helvetica'

# Read data

In [3]:
tpm_fraction_list = ['total', '80S', 'LMW', 'HMW']
pol_fraction_list = ['80S', 'LMW', 'HMW']
timepoint_list = [2, 4, 6, 10]

lib_tpm_col = 'TPM_library'
log2_lib_tpm_col = 'log2_TPM_library'

log2_rrs_cols = [f'log2_RRS_{f}_{t}hpf' for f in pol_fraction_list for t in timepoint_list]

delta_log2_x_cols = [f'Δlog2_X_{t}hpf' for t in timepoint_list]
log2_x_cols = [f'log2_X_{t}hpf' for t in timepoint_list]

In [4]:
# Load data
data_full = pandas.read_csv(
    '../preprocess_data/Zb_5UTR_MPRA_preprocessed.tsv.gz',
    index_col=0,
    sep='\t',
)

data_full

,chr,strand,external_gene_name,utr_length,insert_length,n_uORFs,GC_content,mxfold,index,index_base,...,log2_RRS_HMW_6hpf,log2_RRS_HMW_10hpf,MRL_2hpf,log2_MRL_2hpf,MRL_4hpf,log2_MRL_4hpf,MRL_6hpf,log2_MRL_6hpf,MRL_10hpf,log2_MRL_10hpf
ENSDARG00000000001_ENSDART00000000004_19058_slc35a5_20318,chr9,-,slc35a5,103.0,103.0,2.0,52.427184,23.9,20318.0,20318,...,-1.691783,-1.551375,5.026628,2.329591,6.713248,2.747011,7.002960,2.807965,6.480393,2.696081
ENSDARG00000000018_ENSDART00000181044_14421_nrf1_72681,chr4,-,nrf1,134.0,134.0,0.0,61.940299,35.3,72681.0,72681,...,-0.488586,-1.978301,5.450300,2.446336,6.196041,2.631347,8.359783,3.063465,4.441558,2.151066
ENSDARG00000000019_ENSDART00000124452_14118_ube2h_27446,chr4,+,ube2h,178.0,178.0,1.0,46.629213,30.1,27446.0,27446,...,-0.519735,-0.728637,5.911159,2.563441,10.441205,3.384216,7.626433,2.931009,6.260806,2.646348
ENSDARG00000000068_ENSDART00000000069_2438_slc9a3r1a_113092,chr12,+,slc9a3r1a,152.0,152.0,0.0,46.052632,26.1,113092.0,113092,...,0.006799,-0.418159,14.368484,3.844836,12.294140,3.619899,11.405933,3.511713,9.108052,3.187143
ENSDARG00000000069_ENSDART00000000070_12170_dap_20320,chr24,-,dap,153.0,153.0,1.0,47.058824,31.8,20320.0,20320,...,-0.849952,-2.267783,7.103448,2.828519,6.990700,2.805437,8.646954,3.112192,4.903643,2.293854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSDARG00000025554_ENSDART00000103273_1746_wdr83os_27060,chr11,+,wdr83os,126.0,126.0,1.0,37.301587,22.8,27060.0,27060,...,-0.684112,1.094197,8.956233,3.162892,13.598507,3.765376,8.083156,3.014919,20.048542,4.325425
ENSDARG00000103318_ENSDART00000161570_7325_mrpl3_86762,chr19,+,mrpl3,111.0,111.0,2.0,34.234234,12.5,86762.0,86762,...,-0.736038,-0.117464,11.157909,3.479995,11.045525,3.465390,8.266639,3.047301,12.727341,3.669859
ENSDARG00000036698_ENSDART00000053300_7697_znf865_21263.6,chr19,-,znf865,1305.0,197.0,4.0,31.979695,25.9,21263.6,21263,...,0.440297,-1.271031,13.771432,3.783607,19.339298,4.273464,17.089539,4.095042,7.074025,2.822531
ENSDARG00000056892_ENSDART00000148517_5556_mpp6a_23746.2,chr16,-,mpp6a,311.0,161.0,1.0,39.130435,37.6,23746.2,23746,...,0.382270,-0.355217,13.484335,3.753213,14.144042,3.822123,19.909238,4.315366,16.071784,4.006458


In [5]:
# Load secondary structure
with open('stucture_results_28C.pickle', 'rb') as handle:
    structure_results = pickle.load(handle)

seq_prefix_len = len(structure_results['seq_prefix'])
seq_suffix_len = len(structure_results['seq_suffix'])
structure_res_seq = structure_results['structure_results']

In [6]:
# Add secondary structure info to dataframe
for index, row in data_full.iterrows():
    data_full.loc[index, 'free_energy'] = structure_res_seq[index]['free_energy']
    data_full.loc[index, 'adjusted_free_energy'] = structure_res_seq[index]['free_energy'] / (seq_prefix_len + len(row['insert_seq']) + seq_suffix_len) * 100
    unpaired_prob = structure_res_seq[index]['unpaired_prob'][seq_prefix_len:-seq_suffix_len]
    unpaired_prob_aug = structure_res_seq[index]['unpaired_prob'][-seq_suffix_len: -seq_suffix_len + 3]
    assert(len(data_full.loc[index, 'insert_seq']) == len(unpaired_prob))
    data_full.loc[index, 'unpaired_prob_avg'] = numpy.mean(unpaired_prob)
    data_full.loc[index, 'unpaired_prob_median'] = numpy.median(unpaired_prob)
    data_full.loc[index, 'unpaired_prob_min'] = numpy.min(unpaired_prob)
    data_full.loc[index, 'unpaired_prob_max'] = numpy.max(unpaired_prob)
    data_full.loc[index, 'unpaired_prob_q25'] = numpy.quantile(unpaired_prob, 0.25)
    data_full.loc[index, 'unpaired_prob_q75'] = numpy.quantile(unpaired_prob, 0.75)
    data_full.loc[index, 'unpaired_prob_avg_aug'] = numpy.mean(unpaired_prob_aug)
    

# Plots

In [7]:
# Plot log2_MRL and delta_log2_TPM_total against some metric
def plot_vs_metric(metric_name):
    fig, axes = pyplot.subplots(
        4,
        len(timepoint_list),
        figsize=(3*len(timepoint_list), 3*4),
        sharey='row',
    )

    # Plot log2_RRS
    for fraction_idx, fraction in enumerate(pol_fraction_list):
        for timepoint_idx, timepoint in enumerate(timepoint_list):
            ax = axes[fraction_idx, timepoint_idx]
            ax.set_box_aspect(1)
            custom_plots.plot_scatter_shaded(
                data_full[metric_name],
                data_full[f'log2_RRS_{fraction}_{timepoint}hpf'],
                linreg=True,
                ax=ax,
                rasterized=False,
                s=1,
            )
            if fraction_idx==0:
                ax.set_title(f"t = {timepoint} hpf")
            # ax.set_xlabel('Insert length')
            if timepoint_idx==0:
                ax.set_ylabel(f'log2_RRS_{fraction}')
        # Set the y position of all annotation objects in the row to the maximum y limit
        annotation_objects = [ax.texts[0] for ax in axes[fraction_idx]]
        for annotation in annotation_objects:
            annotation.xy = (annotation.xy[0], ax.get_ylim()[1])


    # axes[1, 0].set_visible(False)
    for timepoint_idx, timepoint in enumerate(timepoint_list):
        ax = axes[3, timepoint_idx]
        ax.set_box_aspect(1)
        custom_plots.plot_scatter_shaded(
            data_full[metric_name],
            data_full[f'Δlog2_X_{timepoint}hpf'],
            linreg=True,
            ax=ax,
            rasterized=False,
            s=1,
        )
        # ax.set_title(f"t = {timepoint} hpf")
        ax.set_xlabel(metric_name)
        if timepoint_idx==0:
            ax.set_ylabel('Δlog2_X')
    # Set the y position of all annotation objects in the row to the maximum y limit
    annotation_objects = [ax.texts[0] for ax in axes[3]]
    for annotation in annotation_objects:
        annotation.xy = (annotation.xy[0], ax.get_ylim()[1])

    return fig

In [12]:
metrics_to_plot = [
    'unpaired_prob_avg',
    # 'unpaired_prob_median',
    # 'unpaired_prob_min',
    # 'unpaired_prob_max',
    # 'unpaired_prob_q25',
    # 'unpaired_prob_q75',
    # 'unpaired_prob_avg_aug',
    'free_energy',
    'adjusted_free_energy',
]
for metric in metrics_to_plot:
    fig = plot_vs_metric(metric)
    fig.suptitle(metric, y=0.93)
    fig.savefig(f'RRS_deltaX_{metric}.pdf', dpi=300, bbox_inches='tight')
    pyplot.close(fig)

In [11]:
# Plot length vs unpaired probabilities
metrics_to_plot = [
    'unpaired_prob_avg',
    # 'unpaired_prob_median',
    # 'unpaired_prob_min',
    # 'unpaired_prob_max',
    # 'unpaired_prob_q25',
    # 'unpaired_prob_q75',
    # 'unpaired_prob_avg_aug',
    'free_energy',
]

for metric_idx, metric in enumerate(metrics_to_plot):

    fig, ax = pyplot.subplots(figsize=(3, 3 ))
    ax.set_box_aspect(1)
    custom_plots.plot_scatter_shaded(
        data_full['insert_seq'].apply(len),
        data_full[metric],
        linreg=True,
        ax=ax,
        rasterized=False,
        s=1,
    )
    ax.set_xlabel('Insert length')
    ax.set_ylabel(metric)
    fig.savefig(f'length_{metric}.pdf', dpi=300, bbox_inches='tight')
    pyplot.close(fig)